In [ ]:
#VERSION SANS PRUNING
#def objective_focused(trial):
#    quant_idx  = trial.suggest_int("quant_idx",  0, len(quantisation_candidates) - 1)
#    mono_idx   = trial.suggest_int("mono_idx",   0, len(monotone_constraint_candidates) - 1)
#    param = {
#        'objective': 'RMSE', 'loss_function': 'RMSE', 'eval_metric': 'RMSE',
#        'random_seed': RANDOM_STATE, 'cat_features':  [c for c in selected_rfecv if c in cat_feats_fs],
#        'verbose': 0, 'early_stopping_rounds': 50,
#
#        'iterations': trial.suggest_int('iterations', int(max(100, center_iterations * (1 - GRID_SEARCH_RANGE_FACTOR*1.5))), int(center_iterations * (1 + GRID_SEARCH_RANGE_FACTOR*1.5))), # Slightly wider range than GS
#        'learning_rate': trial.suggest_float('learning_rate', max(0.001, center_lr * (1 - GRID_SEARCH_RANGE_FACTOR*1.5)), center_lr * (1 + GRID_SEARCH_RANGE_FACTOR*1.5), log=True),
#        'depth': trial.suggest_int('depth', max(2, center_depth - 1), center_depth + 1), # Tight range around best depth
#        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', max(0.01, center_l2 * (1 - GRID_SEARCH_RANGE_FACTOR*1.5)), center_l2 * (1 + GRID_SEARCH_RANGE_FACTOR*1.5), log=True),
#        'border_count': trial.suggest_int('border_count', int(max(32, center_border * (1 - GRID_SEARCH_RANGE_FACTOR*1.5))), int(min(255, center_border * (1 + GRID_SEARCH_RANGE_FACTOR*1.5)))),
#
#        'per_float_feature_quantization': quantisation_candidates[quant_idx],
#        'monotone_constraints':            monotone_constraint_candidates[mono_idx],}
#
#    cv = KFold(n_splits=CV_FOLDS_TUNING, shuffle=True, random_state=RANDOM_STATE)
#    rmses = []
#    for fold, (train_idx, val_idx) in enumerate(cv.split(X_rfecv, y_train)):
#        X_t, X_v = X_rfecv.iloc[train_idx], X_rfecv.iloc[val_idx]
#        y_t, y_v = y_train.iloc[train_idx], y_train.iloc[val_idx]
#        # Optional Oversampling (inside fold)
#        if ENABLE_OVERSAMPLING and HOLIDAY_COLUMN in X_t.columns: pass # Placeholder
#        try:
#            model_cv = CatBoostRegressor(**param)
#            model_cv.fit(X_t, y_t, eval_set=[(X_v, y_v)])
#            preds = model_cv.predict(X_v)
#            fold_rmse = root_mean_squared_error(y_v, preds)
#            rmses.append(fold_rmse)
#        except Exception as e:
#             print(f"Warning: Optuna trial fold {fold+1} failed: {e}")
#             return np.inf 
#    if not rmses: return np.inf
#    return np.mean(rmses)
#
#study_focused = optuna.create_study(direction='minimize')
#print(f"Running Focused Optuna study for {N_TRIALS_OPTUNA_FOCUSED} trials...")
#study_focused.optimize(objective_focused, n_trials=N_TRIALS_OPTUNA_FOCUSED)
#
#print("Focused Optuna Optimization Finished.")
#best_params_optuna_focused = {}
#best_value_optuna_focused = np.inf
#final_model_optuna_focused = None
#optuna_focused_model_metrics = {'RMSE': np.inf, 'MAPE': np.inf, 'R2': -np.inf}
#
#if study_focused.best_trial:
#    trial_focused = study_focused.best_trial
#    best_value_optuna_focused = trial_focused.value
#    print(f"  Value (RMSE): {best_value_optuna_focused:.4f}")
#    print("  Best Parameters Found by Focused Optuna: ")
#    best_params_optuna_focused = trial_focused.params
#
#    print(json.dumps(best_params_optuna_focused, indent=4))
#    with open('best_params_opt_gold_mon.json', "w") as f:
#        json.dump(best_params_optuna_focused,f, indent=4)
#else:
#     print("Focused Optuna study did not complete successfully or found no best trial.")


In [ ]:
#VERSION SANS PRUNING
#with open('best_params_opt_gold_mon.json', "r") as f:
#        best_params_optuna_focused= json.load(f)
#quant_idx = best_params_optuna_focused.pop("quant_idx")
#mono_idx  = best_params_optuna_focused.pop("mono_idx")
#
## 2) map back into your pre-built lists
#best_params_optuna_focused["per_float_feature_quantization"] = quantisation_candidates[quant_idx]
#best_params_optuna_focused["monotone_constraints"]            = monotone_constraint_candidates[mono_idx]
#print(best_params_optuna_focused)
#final_model_optuna_focused = CatBoostRegressor(
#    cat_features=cat_variables, loss_function='RMSE', eval_metric='RMSE', random_seed=RANDOM_STATE, **best_params_optuna_focused)
#final_model_optuna_focused.fit(X_rfecv, y_train, eval_set=(X_rfecv_test, y_test), early_stopping_rounds=200, verbose=0, plot=True)
#final_model_optuna_focused.save_model(
#    "models/focused_optuna_model_no_target.cbm"    # filename
#)
#print("\n--- Focused Optuna Final Model Evaluation ---")
#y_pred_optuna_focused = final_model_optuna_focused.predict(X_rfecv_test)
#rmse_optuna_focused = root_mean_squared_error(y_test, y_pred_optuna_focused)
#mape_optuna_focused = mean_absolute_percentage_error(y_test, y_pred_optuna_focused) * 100
#r2_optuna_focused = r2_score(y_test, y_pred_optuna_focused)
#print(f"Test RMSE: {rmse_optuna_focused:.2f}, MAPE: {mape_optuna_focused:.2f}%, R2: {r2_optuna_focused:.4f}")
#optuna_focused_model_metrics = {'RMSE': rmse_optuna_focused, 'MAPE': mape_optuna_focused, 'R2': r2_optuna_focused}


In [ ]:
#VERSION MULTIPLES NOTEBOOKS
#print("\n" + "="*50)
#print("--- Overall Model Comparison & Analysis (with CV Evaluation) ---")
#
## Imports for CV and metrics
#from sklearn.model_selection import KFold, cross_validate
#from sklearn.metrics import make_scorer, root_mean_squared_error, mean_absolute_percentage_error, r2_score
#from catboost import CatBoostRegressor, Pool, cv as catboost_cv # Ensure Pool is imported
#import numpy as np
#import pandas as pd
#import os # Ensure os is imported
#
#
#
## 2) Define the full list of categorical features (including date)
#cat_features = cats_rfecv
#
## 3) Build a single Pool over your training data with cat_features
#train_pool = Pool(data=X_rfecv, label=y_train, cat_features=cat_features)
#
## --- Define CV strategy ---------------------------------------------
#N_CV_EVAL_FOLDS = 10
#print(f"Using {N_CV_EVAL_FOLDS}-Fold Cross-Validation for model evaluation metrics.")
#
## --- Load models and run CatBoost’s native cv() ---------------------
#models_to_load_paths = {
#    "Baseline": 'models/cb_base_model.cbm',
#    "TimeAware": 'models/cb_time_model.cbm',
#    "Golden":    'models/cb_golden_model.cbm',
#    "RS_tuned":  'models/RS_model_mon_gold.cbm',
#    "optuna":    'models/focused_optuna_model_no_target.cbm',
#    "optuna_prun": 'models/focused_optuna_prun.cbm'
#}
#
#model_cv_metrics = {}
#loaded_models = {}
#for name, path in models_to_load_paths.items():
#    try:
#        # load the model hyper-params
#        m = CatBoostRegressor()
#        m.load_model(path)
#        loaded_models[name] = model
#        print(f"Loaded '{name}', running CatBoost.cv …")
#
#        # grab the params and inject eval_metric + custom_metric
#        params = m.get_params()
#        # primary metric = RMSE (matches your loss_function), extras = MAPE and R2
#        params['eval_metric']   = 'RMSE'
#        params['custom_metric'] = ['MAPE','R2']
#
#        # run the 10-fold CV
#        cv_df = catboost_cv(
#            pool=train_pool,
#            params=params,
#            fold_count=N_CV_EVAL_FOLDS,
#            partition_random_seed=RANDOM_STATE,
#            shuffle=True,
#            as_pandas=True,
#            verbose=False
#        )
#        # cv_df columns include:
#        #   'iterations',
#        #   'test-RMSE-mean', 'test-RMSE-std',
#        #   'test-MAPE-mean','test-MAPE-std',
#        #   'test-R2-mean','test-R2-std', etc.
#
#        # 2) Pull out the final means from the last row:
#        mean_rmse = cv_df['test-RMSE-mean'].iloc[-1]
#        mean_mape = cv_df['test-MAPE-mean'].iloc[-1] * 100     # percent
#        mean_r2   = cv_df['test-R2-mean'].iloc[-1]
#
#        model_cv_metrics[name] = {
#            'RMSE': mean_rmse,
#            'MAPE': mean_mape,
#            'R2':   mean_r2
#        }
#        print(f"  → {name}: RMSE={mean_rmse:.4f}, MAPE={mean_mape:.2f}%, R²={mean_r2:.4f}")
#
#    except Exception as e:
#        print(f"Error evaluating '{name}': {e}. Skipping.")
#
#
#print("\nCombining all models for comparison...")
#all_models_to_compare = loaded_models.copy() 
#
## --- Create Metrics DataFrame (using MEAN CV metrics) ---
#print("\nCreating final metrics comparison table using Mean CV Scores...")
#if not model_cv_metrics:
#     print("Warning: No model CV metrics available for comparison table.")
#     comparison_df_all = pd.DataFrame() # Empty DataFrame
#else:
#    # build DataFrame and keep only RMSE/ MAP E/ R2
#    comparison_df_all = pd.DataFrame(model_cv_metrics).T
#    cols_to_show = [c for c in ['RMSE','MAPE','R2'] if c in comparison_df_all.columns]
#    comparison_df_all = comparison_df_all[cols_to_show]
#    print("\nCombined Model Performance Comparison (Mean CV Scores on Training Data):")
#    print(comparison_df_all.sort_values('RMSE').to_string(float_format="%.4f"))
#    
#    # save JSON
#    METRICS_FILENAME = 'final_metrics.json'
#    print(f"Saving final CV metrics comparison to {METRICS_FILENAME}...")
#    comparison_df_all.to_json(METRICS_FILENAME, indent=4, orient="index")
#
## --- Run CatBoost `.compare()` on the TEST SET ---
## Note: .compare() still uses the single test split for direct comparison
#print("\n--- Comparing CatBoost Models using model.compare() on Test Set ---")
#
#    # Create comparison pool using the original TEST set
#
#X_rfecv_test['date'] = X_rfecv_test['date'].astype(str)
#comparison_pool = Pool(data=X_rfecv_test, label=y_test, cat_features=cat_features)
#model_names = list(all_models_to_compare.keys())
#for i in range(len(model_names)):
#    for j in range(i+1, len(model_names)):
#        n1, n2 = model_names[i], model_names[j]
#        m1 = all_models_to_compare[n1]
#        m2 = all_models_to_compare[n2]
#        print(f"\nComparing '{n1}' vs '{n2}' on Test Set…")
#        try:
#            comp_df = m1.compare(
#                m2,
#                comparison_pool,
#                metrics=['RMSE']     # you can also add 'MAPE','R2' here if desired
#            )
#            print(comp_df[['metric_name','model1_wins','model2_wins','metric_value_diff']])
#        except Exception as e:
#            print(f"  Could not compare {n1} vs {n2}: {e}")
#print("="*50 + "\n")
#
#
## --- Final Model Analysis (Choose Overall Best based on CV Metrics Table) ---
#print("\n--- Final Model Analysis (Overall Best based on CV Metrics) ---")
#final_model_to_analyze = None
#best_model_name = None
#
## Use the comparison_df_all which now contains CV metrics
#if not comparison_df_all.empty:
#     try:
#        best_name = comparison_df_all['RMSE'].idxmin()
#        print(f"Selected '{best_name}' as best model (lowest Mean CV RMSE).")
#        final_model_to_analyze = all_models_to_compare[best_name]
#     except Exception as e:
#          print(f"Could not determine best model from CV metrics table: {e}")
#
## Analyze the selected best model (trained on full training data)
#if final_model_to_analyze and final_model_to_analyze.is_fitted():
#    print(f"\nAnalyzing model: {best_model_name}")
#    # Feature Importance (using the model trained on full X_rfecv)
#    print("\nPlotting native feature importance...")
#    if 'X_rfecv' in locals() and hasattr(X_rfecv, 'columns'):
#        final_feature_names = X_rfecv.columns.tolist()
#        # Assuming plot_native_feature_importance is available from utils
#        # You might want to disable plotting in Azure ML jobs or save plots to output dir
#        try:
#             from utils import plot_native_feature_importance
#             # plot_native_feature_importance(final_model_to_analyze, final_feature_names) # Comment out plt.show() in utils for Azure ML
#             print("Native importance plot generation skipped for Azure ML script.")
#        except ImportError:
#             print("Could not import plot_native_feature_importance from utils.")
#    else:
#        print("Warning: Cannot get feature names for native importance plot.")
#
#    # SHAP Analysis (Optional - can be computationally expensive)
#    print("\nGenerating SHAP plots...")
#    if 'X_rfecv_test' in locals() and 'final_feature_names' in locals():
#        try:
#            from utils import plot_shap_summary, plot_shap_waterfall
#            plot_shap_summary(final_model_to_analyze, X_rfecv_test, final_feature_names, plot_type="bar") # Comment out plt.show()
#            plot_shap_summary(final_model_to_analyze, X_rfecv_test, final_feature_names, plot_type="dot") # Comment out plt.show()
#            plot_shap_waterfall(final_model_to_analyze, X_rfecv_test, final_feature_names, instance_index=1) # Comment out plt.show()
#            print("SHAP plot generation skipped for Azure ML script.")
#        except ImportError:
#             print("Could not import SHAP plotting functions from utils.")
#    else:
#        print("Warning: Cannot generate SHAP plots (X_rfecv_test or feature names not available?).")
#else:
#    print("No final best model available for analysis (either none found or selection failed).")
#
#print("\n" + "="*50)
#print("Script section finished.")
## %%
#

In [ ]:
'''# pondérer différentes stations car sinon non-représentatif
# soit prendre individuellement chacune variables climatiques, soit pondérer
# à voir avec la feature selection

from meteostat import Point, Daily
import pandas as pd, os, joblib

CACHE_FILE = "data/_wx_cache.joblib"

def fetch_station_series(lat=48.8566, lon=2.3522): # station de Paris
    if os.path.exists(CACHE_FILE):
        daily = joblib.load(CACHE_FILE)
        daily.index = daily.index.tz_localize(None)
        daily.index.name = 'date'
        return daily.reset_index()

    start = pd.to_datetime(df['date']).min().to_pydatetime().replace(tzinfo=None)
    end   = pd.to_datetime(df['date']).max().to_pydatetime().replace(tzinfo=None)

    point = Point(lat, lon)
    daily = Daily(point, start=start, end=end).fetch()

    keep = [c for c in ['wspd','tmin','tmax'] if c in daily.columns]
    daily = daily[keep].rename(columns={'wspd':'windspeed'})

    if 'tmin' in daily.columns and 'tmax' in daily.columns:
        daily['t_range'] = daily['tmax'] - daily['tmin']
        daily = daily.drop(columns=['tmin','tmax'])

    daily.index = daily.index.tz_localize(None)
    daily.index.name = 'date'

    joblib.dump(daily, CACHE_FILE)
    return daily.reset_index()

# re-fetch & merge
wx = fetch_station_series()

df['date'] = pd.to_datetime(df['date']).dt.normalize().dt.tz_localize(None)

wx['date'] = pd.to_datetime(wx['date']).dt.normalize().dt.tz_localize(None)

df = (df.merge(wx, on='date', how='left').assign(hdd_wind=lambda x: x['hdd'] * (1 + x['windspeed']/10)))'''